# ReAct con LangGraph: Arquitectura de Agente con Herramientas Visualizable

Este notebook demuestra como migrar ReAct (Reasoning + Acting) desde implementaciones manuales de LangChain a **LangGraph**, habilitando:

1. **Visualizacion de arquitectura**: Grafos PNG renderizados con `draw_mermaid_png()`
2. **ToolNode integrado**: Ejecucion automatica de herramientas con routing
3. **Debugging mejorado**: Trazas de tool calls con observaciones
4. **Patron agente-tools**: Ciclo automatizado Thought->Action->Observation
5. **Composabilidad**: Nodos reutilizables y flujos condicionales

---

## Fundamento Teorico: ReAct (Reasoning + Acting)

### Origen y Contexto Historico

ReAct es un paradigma de prompting introducido por **Yao et al. (2022)** en el paper "ReAct: Synergizing Reasoning and Acting in Language Models". A diferencia de Chain of Thought (CoT) que solo genera razonamiento interno, ReAct permite que el modelo **interactue con el mundo externo** mediante herramientas.

El fundamento teorico se basa en:

1. **Agentes cognitivos** (Newell & Simon, 1976): Los sistemas inteligentes requieren tanto razonamiento interno como capacidad de actuar sobre el entorno.
2. **Aprendizaje activo** (Sutton & Barto, 1998): La observacion de resultados de acciones mejora la toma de decisiones futuras.
3. **Tool use en LLMs** (Schick et al., 2023 - Toolformer): Los modelos pueden aprender cuando y como usar herramientas externas.

### Diseno Arquitectonico

En esta implementacion, ReAct se estructura como un **grafo ciclico** con los siguientes componentes:

```
agent -> [tools] -> agent -> [tools] -> ... -> END
   |                 |         |
Thought           Action   Observation
```

**Componentes clave:**

1. **Agent Node**: Decide que accion tomar (call tool o dar respuesta final)
2. **Tool Node**: Ejecuta herramientas en paralelo y retorna observaciones
3. **Conditional Router**: Decide si continuar con tools o terminar
4. **Message History**: Mantiene historial completo de Thought->Action->Observation

**Ciclo ReAct:**

- **Thought**: El agente razona sobre el estado actual y decide que hacer
- **Action**: Ejecuta una herramienta especifica (analizar_perfil, generar_mensaje, auditar_respeto)
- **Observation**: Recibe resultado de la herramienta y actualiza su conocimiento
- **Repeat**: Vuelve a Thought con nueva informacion hasta completar tarea

### Criterios de Decision

**Usa ReAct cuando:**

- Necesitas interactuar con APIs, bases de datos, o sistemas externos
- La tarea requiere validacion o auditoria de outputs intermedios
- Quieres trazabilidad completa de acciones (compliance, debugging)
- El razonamiento debe estar fundamentado en observaciones reales (no alucinaciones)
- Necesitas control granular sobre el flujo de ejecucion

**NO uses ReAct cuando:**

- La tarea es puramente de razonamiento interno (usa CoT)
- No tienes herramientas que aporten valor real
- La latencia es critica y las herramientas son lentas
- Una llamada directa al LLM ya cumple los requisitos
- El costo de tool calls no justifica la mejora de precision

### Trade-offs y Limitaciones

| Dimension | ReAct | CoT | Direct Prompting | Notas |
|-----------|-------|-----|------------------|-------|
| **Precision con tools** | Muy Alta | Media | Baja | ReAct evita alucinaciones usando tools |
| **Latencia** | Alta (tools + LLM) | Media (solo LLM) | Baja | Cada tool call agrega latencia |
| **Costo** | Alto (multi-turn) | Medio | Bajo | ~5-10x mas tokens que direct |
| **Trazabilidad** | Maxima | Alta | Baja | Historial completo de actions |
| **Explicabilidad** | Maxima | Alta | Baja | Thought + Action + Observation |
| **Control de flujo** | Dinamico | Fijo | Fijo | El agente decide dinamicamente |
| **Complejidad impl.** | Alta | Media | Baja | Requiere tools + routing |
| **Failure modes** | Tool errors | Logic errors | Hallucinations | Diferentes puntos de falla |

**Failure modes comunes:**

1. **Tool description drift**: Descripciones ambiguas -> el agente usa tools incorrectamente
2. **Infinite loops**: Sin guardrails de max_iterations -> el agente cicla indefinidamente
3. **Tool errors no manejados**: Herramienta falla -> todo el pipeline se rompe
4. **Hallucination in observations**: El agente inventa resultados de tools que no ejecuto
5. **Tool overload**: Demasiadas herramientas -> el agente se confunde

### Consideraciones de Produccion

**Escalabilidad:**
- ReAct es inherentemente secuencial (cada tool call depende del anterior)
- Considera ejecutar tools independientes en paralelo (ToolNode hace esto automaticamente)
- Implementa caching de tool results para contextos similares
- Usa modelos mas rapidos para decision making (Haiku), mas potentes para razonamiento complejo (Opus)

**Monitoreo:**
- Trackea: num_tool_calls, latency_per_tool, token_usage_per_turn, success_rate
- Alertas: max_iterations_exceeded, tool_error_rate > 5%, unexpected_tool_sequences
- Logs: Historial completo de messages para replay y debugging

**Timeouts:**
- Timeout por tool (e.g., 5s para tools rapidos, 30s para APIs lentas)
- Timeout global del agente (e.g., 2 minutos)
- Max iterations (e.g., 10 tool calls) para evitar loops infinitos

**Validacion de tools:**
- Todas las tools deben tener docstrings claros con ejemplos
- Input/output schemas bien definidos (Pydantic)
- Error handling explicito en cada tool
- Unit tests para cada tool antes de usarlos en el agente

### Referencias Academicas

- **Yao et al. (2022)**: "ReAct: Synergizing Reasoning and Acting in Language Models" - Paper original de ReAct
- **Schick et al. (2023)**: "Toolformer: Language Models Can Teach Themselves to Use Tools" - Fundamento de tool use
- **Newell & Simon (1976)**: "Computer Science as Empirical Inquiry" - Fundamento cognitivo de agentes
- **Sutton & Barto (1998)**: "Reinforcement Learning: An Introduction" - Learning from observations
- **Wei et al. (2022)**: "Chain of Thought Prompting" - Comparacion con CoT puro

---

In [1]:
# Setup
import sys
from pathlib import Path

# Agregar root al path para imports
repo_root = Path.cwd()
while not (repo_root / "pyproject.toml").exists() and repo_root != repo_root.parent:
    repo_root = repo_root.parent
sys.path.insert(0, str(repo_root))

# Agregar 03_langchain_prompting al path para common imports
langchain_prompting_path = repo_root / "03_langchain_prompting"
if str(langchain_prompting_path) not in sys.path:
    sys.path.insert(0, str(langchain_prompting_path))

# Imports
from IPython.display import Image, display
import json

In [2]:
# Importar modulo
from ReAct_LangChain.Notebooks import react_langgraph_02 as react_module

# Importar funciones
build_react_graph = react_module.build_react_graph
run_react_langgraph = react_module.run_react_langgraph
ReActState = react_module.ReActState

ModuleNotFoundError: No module named 'ReAct_LangChain'

## Visualizacion del Grafo ReAct

A continuacion, construimos el grafo de ReAct y visualizamos su arquitectura usando `draw_mermaid_png()`. Este grafo muestra:

- **Agent Node**: Decide que accion tomar (thought)
- **Tool Node**: Ejecuta herramientas en paralelo (action)
- **Conditional Routing**: Decide si continuar con tools o terminar
- **Ciclo feedback**: tools -> agent -> tools (observation loop)

**Nota tecnica**: LangGraph renderiza el grafo usando Mermaid y Graphviz. Si la visualizacion falla, se mostrara el codigo Mermaid en texto plano.

**Diferencia clave vs CoT**: En CoT el flujo es lineal (zero_shot -> few_shot -> evaluate -> refine). En ReAct el flujo es **ciclico** (agent <-> tools) hasta que el agente decide terminar.

In [ ]:
# Construir grafo ReAct
app = build_react_graph()

# Visualizar arquitectura
try:
    display(Image(app.get_graph().draw_mermaid_png()))
    print("\n[Grafo renderizado exitosamente]")
    print("Observa el ciclo: agent -> tools -> agent -> tools -> ... -> END")
except Exception as e:
    print(f"No se pudo renderizar grafo PNG: {e}")
    print("Asegurate de tener graphviz instalado: brew install graphviz (macOS)")
    print("\nGrafo en formato mermaid:")
    print(app.get_graph().draw_mermaid())

## Ejemplo 1: Arquitecta Apasionada por Fotografia Urbana

Ejecutamos el pipeline completo de ReAct con el perfil default. Observa:

1. **Tool Call 1: analizar_perfil** - Extrae insights del context packet
2. **Tool Call 2: generar_mensaje** - Crea opener y follow_up basado en analisis
3. **Tool Call 3: auditar_respeto** - Valida que el mensaje sea respetuoso
4. **Final Answer** - Respuesta estructurada con trace summary

**Trace de ejecucion:**
```
[Agent] Thought: Necesito analizar el perfil primero
[Agent] Action: analizar_perfil(context_packet)
[Tool] Observation: {persona: "arquitecta...", insights: [...]}
[Agent] Thought: Ahora puedo generar el mensaje
[Agent] Action: generar_mensaje(analysis)
[Tool] Observation: {opener: "...", follow_up: "..."}
[Agent] Thought: Debo auditar respeto
[Agent] Action: auditar_respeto(message)
[Tool] Observation: {ok: true, flags: []}
[Agent] Thought: Tengo toda la info, puedo dar respuesta final
[Agent] Final Answer: {...}
```

In [ ]:
# Ejecutar con perfil default
print("=" * 80)
print("EJECUCION: Perfil arquitecta + fotografia urbana")
print("=" * 80)

result = run_react_langgraph(verbose=True)

## Analisis de Resultados Intermedios

Inspeccionamos los resultados de cada herramienta ejecutada durante el ciclo ReAct:

In [ ]:
print("=== ANALISIS DE PERFIL ===")
print(json.dumps(result["analysis"], indent=2, ensure_ascii=False))

print("\n=== DRAFT GENERADO ===")
print(json.dumps(result["draft"], indent=2, ensure_ascii=False))

print("\n=== AUDITORIA DE RESPETO ===")
print(json.dumps(result["audit"], indent=2, ensure_ascii=False))

print("\n=== RESPUESTA FINAL ===")
print(json.dumps(result["final_answer"], indent=2, ensure_ascii=False))

## Trace Summary: Observabilidad Completa

Una de las principales ventajas de ReAct es la **trazabilidad completa** del proceso de decision. Cada tool call queda registrado con su input y output.

In [ ]:
print("=== TRACE DE TOOL CALLS ===")
for i, step in enumerate(result["trace"], 1):
    print(f"\n{i}. Tool: {step['tool']}")
    print(f"   Result preview: {step['result'][:150]}...")

print("\n=== METADATA ===")
print(f"Modelo: {result['__model']}")
print(f"Arquitectura: {result['__architecture']}")
print(f"Context hash: {result['__context_hash']}")

## Errores Comunes al Implementar ReAct

### 1. Tool Descriptions Ambiguas

**Problema**: Docstrings poco claros -> el agente no sabe cuando usar cada tool.

**Malo:**
```python
@tool
def procesar(data: str) -> str:
    """Procesa datos."""
    # Que hace "procesar"? Cuando usarlo?
    return result
```

**Bueno:**
```python
@tool
def analizar_perfil(context_packet_json: str) -> str:
    """Extrae insights accionables desde un context packet JSON.
    
    Usa esta herramienta PRIMERO antes de generar mensajes.
    Input: JSON string con perfil, gustos, estilo.
    Output: JSON con persona, estilo_preferido, insights.
    
    Ejemplo input: {"profile": {"tipo_persona": "arquitecta", ...}}
    Ejemplo output: {"persona": "arquitecta", "insights": [...]}
    """
    ...
```

### 2. Sin Limites de Iteraciones

**Problema**: El agente cicla indefinidamente si no logra completar la tarea.

**Solucion**: Implementar max_iterations y timeout:
```python
def agent_node(state: ReActState) -> ReActState:
    # Validar max iterations
    if state.get("iteration_count", 0) >= 10:
        raise RuntimeError("Max iterations exceeded")
    
    # Incrementar contador
    state["iteration_count"] = state.get("iteration_count", 0) + 1
    ...
```

### 3. No Validar Outputs de Tools

**Problema**: Confias ciegamente en tool outputs sin validar formato/contenido.

**Solucion**: Validacion de schema:
```python
@tool
def analizar_perfil(context_packet_json: str) -> str:
    try:
        context_packet = json.loads(context_packet_json)
        assert "profile" in context_packet
        # ... procesamiento ...
        return json.dumps(result)
    except Exception as e:
        return json.dumps({"error": str(e), "suggestion": "Verifica formato JSON"})
```

### 4. Sobrecarga de Herramientas (Tool Overload)

**Problema**: Dar 20+ tools al agente -> confusion y mal uso.

**Regla empirica**: 3-7 tools es optimo. Mas alla, el agente pierde precision.

**Solucion**: Agrupar tools relacionados:
```python
# Malo: 10 tools micro
analizar_nombre, analizar_gustos, analizar_estilo, ...

# Bueno: 1 tool con opciones
@tool
def analizar_perfil(aspect: Literal["nombre", "gustos", "estilo", "todo"]) -> str:
    ...
```

### 5. Sin Timeout por Tool

**Problema**: Una tool lenta bloquea todo el pipeline.

**Solucion**: Wrapper con timeout:
```python
from functools import wraps
import signal

def timeout(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            def handler(signum, frame):
                raise TimeoutError(f"Tool {func.__name__} timeout after {seconds}s")
            
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(seconds)
            try:
                return func(*args, **kwargs)
            finally:
                signal.alarm(0)
        return wrapper
    return decorator

@tool
@timeout(5)  # Max 5 segundos
def slow_tool(data: str) -> str:
    ...
```

### 6. Sin Guardrails de Secuencia

**Problema**: El agente ejecuta tools en orden incorrecto (e.g., auditar antes de generar).

**Solucion**: Validacion de secuencia en agent node:
```python
def agent_node(state: ReActState) -> ReActState:
    # Validar secuencia esperada
    executed_tools = [msg.name for msg in state["messages"] if isinstance(msg, ToolMessage)]
    
    if "generar_mensaje" in executed_tools and "analizar_perfil" not in executed_tools:
        raise RuntimeError("Debes analizar perfil ANTES de generar mensaje")
    ...
```

---

## Decision Tree: Cuando Usar ReAct

```
Necesitas interactuar con sistemas externos (APIs, DBs, validadores)?
- SI NO → Evalua: Necesitas razonamiento multi-paso?
  - SI SI → Usa CoT (mas simple, mas rapido)
  - SI NO → Usa Direct Prompting
- SI SI → Evalua: Las tools aportan valor real (no mock)?
    - SI NO → Reconsiderar arquitectura (tal vez no necesitas agente)
    - SI SI → Evalua: El costo/latencia es aceptable?
        - SI NO → Optimizar tools (caching, paralelizacion) o reconsiderar
        - SI SI → Usa ReAct
```

**Casos de uso claros para ReAct:**

- **Customer support con acceso a CRM**: Buscar info del cliente, crear tickets, actualizar estado
- **Data analysis agent**: Query databases, run scripts, generate reports
- **Code review bot**: Read files, run linters, suggest fixes, create PRs
- **Content moderation**: Analyze text, check against policies, flag/approve
- **Booking assistant**: Check availability, make reservations, send confirmations

**Casos donde ReAct es overkill:**

- **Clasificacion de texto**: No necesitas tools, un prompt directo funciona
- **Traduccion simple**: Direct prompting es suficiente
- **Resumen de texto**: CoT puede ayudar, pero tools no aportan
- **Generacion creativa**: Las tools suelen limitar creatividad

---

## Comparacion: LangGraph vs Loop Manual

### Implementacion Manual (Loop ReAct basico)

```python
# ReAct manual con while loop
messages = [HumanMessage(content="Task: ...")]
max_iterations = 10

for iteration in range(max_iterations):
    # Agent decide
    response = llm_with_tools.invoke(messages)
    messages.append(response)
    
    # Si tiene tool calls, ejecutarlos
    if hasattr(response, "tool_calls") and response.tool_calls:
        for tool_call in response.tool_calls:
            # Buscar tool manualmente
            tool = next(t for t in tools if t.name == tool_call["name"])
            
            # Ejecutar tool
            result = tool.invoke(tool_call["args"])
            
            # Agregar observacion
            messages.append(ToolMessage(
                content=result,
                name=tool_call["name"],
                tool_call_id=tool_call["id"]
            ))
    else:
        # No mas tool calls, terminar
        break

final_answer = messages[-1].content
```

**Problemas:**
- Estado (messages) pasado manualmente
- Logica de routing mezclada con ejecucion
- Dificil de visualizar
- Sin checkpointing ni recovery
- Dificil de unit test
- No hay paralelizacion de tools

### LangGraph con ToolNode

```python
# ReAct con LangGraph
def agent_node(state: ReActState) -> ReActState:
    llm_with_tools = llm.bind_tools(tools)
    result = llm_with_tools.invoke(state["messages"])
    return {**state, "messages": [result]}

def should_continue(state: ReActState) -> Literal["tools", "end"]:
    last = state["messages"][-1]
    return "tools" if hasattr(last, "tool_calls") and last.tool_calls else "end"

# Construir grafo
graph = StateGraph(ReActState)
graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(tools))  # ToolNode maneja ejecucion automatica

graph.set_entry_point("agent")
graph.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
graph.add_edge("tools", "agent")

app = graph.compile()
result = app.invoke(initial_state)
```

**Ventajas:**
- Estado compartido via StateGraph (menos boilerplate)
- Routing declarativo (should_continue es puro)
- ToolNode ejecuta tools automaticamente **en paralelo** si son independientes
- Visualizacion built-in (draw_mermaid_png)
- Checkpointing con memory/redis (recovery en caso de fallo)
- Mejor testing (cada nodo es unit-testable)
- Streaming de updates en tiempo real

**Trade-off:**
- Setup inicial mas verboso (~40 lineas vs 20 del loop)
- Curva de aprendizaje (conceptos de grafo, state management)
- Overhead minimo en runtime (~5-10ms por invocation)

**Conclusion**: Para prototipos rapidos, loop manual puede ser suficiente. Para produccion o pipelines complejos (3+ tools, conditional routing, checkpointing), LangGraph es superior.

---

## Beneficios del ToolNode

**ToolNode** es una abstraccion de LangGraph que maneja la ejecucion de herramientas automaticamente. Ventajas:

1. **Ejecucion paralela**: Si el agente hace 3 tool calls independientes, ToolNode los ejecuta en paralelo
2. **Error handling**: Captura excepciones y las convierte en ToolMessages con error info
3. **Tool matching**: Busca la tool correcta por nombre automaticamente
4. **Message formatting**: Formatea outputs como ToolMessages con metadata correcto

**Comparacion con ejecucion manual:**

```python
# Manual: ejecutar tools secuencialmente
for tool_call in response.tool_calls:
    tool = next(t for t in tools if t.name == tool_call["name"])
    result = tool.invoke(tool_call["args"])  # Secuencial!
    messages.append(ToolMessage(...))

# ToolNode: ejecutar en paralelo automaticamente
tool_node = ToolNode(tools)
result = tool_node.invoke(state)  # Paralelo si tools son independientes!
```

**Ejemplo de paralelizacion:**

Si el agente decide hacer:
```
tool_calls = [
    {"name": "buscar_usuario", "args": {"id": 123}},
    {"name": "buscar_pedidos", "args": {"user_id": 123}},
    {"name": "buscar_reviews", "args": {"user_id": 123}}
]
```

ToolNode ejecutara las 3 en paralelo (si son independientes) -> latencia total = max(latencias), no sum(latencias).

---

## Proximos Pasos y Experimentacion

### 1. Agrega Herramientas Propias

Experimenta agregando tools relevantes para tu dominio:

```python
@tool
def buscar_documentacion(query: str) -> str:
    """Busca en documentacion interna.
    
    Args:
        query: Termino de busqueda
    
    Returns:
        JSON con resultados relevantes
    """
    # Implementar busqueda real (vector DB, Elasticsearch, etc)
    results = search_docs(query)
    return json.dumps(results)

# Agregar a la lista de tools
tools = [analizar_perfil, generar_mensaje, auditar_respeto, buscar_documentacion]
```

### 2. Implementa Guardrails Sofisticados

Agrega validacion de secuencia, timeouts, y limites:

```python
def agent_node_with_guardrails(state: ReActState) -> ReActState:
    # Validar max iterations
    iteration = state.get("iteration", 0)
    if iteration >= 10:
        return {**state, "messages": [AIMessage(content="Max iterations exceeded")]}
    
    # Validar secuencia de tools
    executed_tools = [m.name for m in state["messages"] if isinstance(m, ToolMessage)]
    if "generar_mensaje" in executed_tools and "analizar_perfil" not in executed_tools:
        return {**state, "messages": [AIMessage(content="Debes analizar antes de generar")]}
    
    # Ejecutar agent normalmente
    result = agent_node(state)
    result["iteration"] = iteration + 1
    return result
```

### 3. Agrega Checkpointing para Long-Running Agents

Para agentes que corren por mucho tiempo, implementa checkpointing:

```python
from langgraph.checkpoint.memory import MemorySaver

# Crear checkpointer
memory = MemorySaver()

# Compilar con checkpointing
app = graph.compile(checkpointer=memory)

# Ejecutar con thread_id para recovery
config = {"configurable": {"thread_id": "user-123-session-456"}}
result = app.invoke(initial_state, config=config)

# Si falla, recuperar estado
checkpoint = memory.get(config)
recovered_result = app.invoke(checkpoint.state, config=config)
```

### 4. Implementa A/B Testing

Compara diferentes versiones del agente:

```python
# Version A: modelo rapido + 3 tools
def agent_node_v1(state):
    llm = ChatOpenAI(model="gpt-4o-mini")
    tools = [analizar_perfil, generar_mensaje, auditar_respeto]
    ...

# Version B: modelo lento + 5 tools
def agent_node_v2(state):
    llm = ChatOpenAI(model="gpt-4o")
    tools = [analizar_perfil, generar_mensaje, auditar_respeto, buscar_docs, validate_output]
    ...

# Medir precision, latencia, costo
results_v1 = [run_react_langgraph(agent_node=agent_node_v1) for _ in range(100)]
results_v2 = [run_react_langgraph(agent_node=agent_node_v2) for _ in range(100)]

# Comparar metricas
print(f"V1 latency: {mean([r['latency'] for r in results_v1])}s")
print(f"V2 latency: {mean([r['latency'] for r in results_v2])}s")
```

### 5. Migra a Async para Mejor Performance

Para produccion, usa invocacion async:

```python
import asyncio

async def agent_node_async(state: ReActState) -> ReActState:
    llm_with_tools = llm.bind_tools(tools)
    result = await llm_with_tools.ainvoke(state["messages"])
    return {**state, "messages": [result]}

# Ejecutar multiples requests en paralelo
results = await asyncio.gather(
    app.ainvoke(state1),
    app.ainvoke(state2),
    app.ainvoke(state3),
)
```

---

## Conclusion y Autocritica

**Ventajas de ReAct con LangGraph:**

1. Visualizacion de arquitectura (grafo PNG) facilita debugging
2. ToolNode elimina boilerplate de ejecucion manual de tools
3. Routing declarativo hace el flujo mas claro y testeable
4. Checkpointing built-in permite recovery en caso de fallos
5. Mejor observabilidad (historial completo de messages)

**Limitaciones y consideraciones:**

1. **Overhead de setup**: LangGraph requiere mas codigo inicial que un loop simple
2. **Curva de aprendizaje**: Conceptos de StateGraph, nodos, edges no son inmediatos
3. **Latencia**: Cada tool call agrega latencia (API round-trip)
4. **Costo**: Multi-turn conversations consumen mas tokens
5. **Debugging**: Aunque mejorado vs manual, debuggear agentes sigue siendo complejo

**Cuando vale la pena la inversion:**

- Pipelines de produccion con requisitos de observabilidad
- Agentes complejos con 3+ tools y conditional routing
- Necesidad de checkpointing (long-running agents)
- Equipos grandes que necesitan visualizacion de arquitectura

**Cuando NO vale la pena:**

- Prototipos rapidos (usa loop manual)
- Agentes simples con 1-2 tools
- Casos donde latencia/costo es critico y precision no justifica el overhead

**Siguiente notebook**: Implementacion de ReAct para casos de uso avanzados (multi-agent systems, human-in-the-loop).

---

---

## Ejemplo 2: Herramientas Especializadas de Coqueteo

Ahora demostramos ReAct con **4 herramientas avanzadas** específicas para coqueteo y matching:

1. **analizar_compatibilidad**: Evalúa match entre perfiles (gustos, estilos, red flags)
2. **generar_icebreaker**: Crea 3 opciones calibradas por tono (conservador/medio/atrevido)
3. **predecir_respuesta**: Predice probabilidad de respuesta + mejoras sugeridas
4. **escalar_conversacion**: Sugiere próximo paso (profundizar/proponer_cita/mantener_ritmo/retomar)

**Objetivo pedagógico**: Mostrar que ReAct puede usar herramientas especializadas de dominio para tomar decisiones más informadas.

In [ ]:
# Importar personas y herramientas para el ejemplo 2
from common.coqueteo_personas import (
    get_persona_romantico_clasico,
    get_persona_moderno_carismatico,
    get_match_cientifica_aventurera
)

# Importar herramientas para inspección
from common.coqueteo_tools import COQUETEO_TOOLS

print("Herramientas especializadas disponibles:")
for tool in COQUETEO_TOOLS:
    print(f"  - {tool.name}: {tool.description[:80]}...")

### 2A. El Romántico Clásico con Herramientas Avanzadas

**Perfil**: Galán mexicano tradicional

**Match**: Neurocientífica aventurera

**Hipótesis**: Las herramientas detectarán:
- Compatibilidad media-alta (intereses complementarios)
- Recomendarán tono conservador para icebreaker
- Identificarán que lenguaje poético puede funcionar con perfil intelectual

In [ ]:
# Obtener personas
persona_romantico = get_persona_romantico_clasico()
match_cientifica = get_match_cientifica_aventurera()

print("=" * 80)
print("ESCENARIO: Romántico Clásico -> Neurocientífica Aventurera")
print("=" * 80)
print(f"Persona: {persona_romantico['pais_origen']} - {persona_romantico['tipo_persona']}")
print(f"Match: {match_cientifica['nombre']} - {match_cientifica['profesion']}")
print(f"\nIntereses del match: {', '.join(match_cientifica['intereses'][:3])}")
print("\n")

# Ejecutar ReAct con herramientas especializadas
result_romantico_tools = run_react_with_coqueteo_tools(
    persona_dict=persona_romantico,
    match_dict=match_cientifica,
    verbose=True
)

#### Análisis: Herramientas en Acción

Inspecciona qué herramientas se llamaron y sus outputs:

1. **analizar_compatibilidad**: ¿Qué porcentaje de compatibilidad detectó? ¿Qué áreas de conexión identificó?
2. **generar_icebreaker**: ¿Qué tono recomendó? ¿Las opciones reflejan el estilo romántico clásico?
3. **predecir_respuesta**: ¿Qué probabilidad de respuesta estimó? ¿Qué mejoras sugirió?

**Observación clave**: Las herramientas permiten que el agente tome decisiones informadas en vez de generar directamente.

In [ ]:
print("\n" + "=" * 80)
print("INSPECCION DETALLADA: Resultados de Herramientas")
print("=" * 80)

tool_results = result_romantico_tools['tool_results']

# 1. Compatibilidad
if 'analizar_compatibilidad' in tool_results:
    compat = tool_results['analizar_compatibilidad']
    print(f"\n1. COMPATIBILIDAD: {compat.get('compatibilidad_porcentaje', 'N/A')}%")
    print(f"   Areas de conexion:")
    for area in compat.get('areas_conexion', [])[:3]:
        print(f"     - {area}")
    print(f"   Red flags: {len(compat.get('red_flags_detectadas', []))}")

# 2. Icebreaker
if 'generar_icebreaker' in tool_results:
    ice = tool_results['generar_icebreaker']
    print(f"\n2. ICEBREAKER GENERADO")
    for i, opcion in enumerate(ice.get('opciones', [])[:2], 1):
        print(f"   Opcion {i}: {opcion['texto'][:100]}...")
        print(f"   Probabilidad exito: {opcion['probabilidad_exito']}%")

# 3. Predicción
if 'predecir_respuesta' in tool_results:
    pred = tool_results['predecir_respuesta']
    print(f"\n3. PREDICCION DE RESPUESTA: {pred.get('probabilidad_respuesta', 'N/A')}%")
    print(f"   Factores positivos: {len(pred.get('factores_positivos', []))}")
    print(f"   Factores negativos: {len(pred.get('factores_negativos', []))}")
    print(f"   Mejoras sugeridas: {len(pred.get('mejoras_sugeridas', []))}")

print("\n" + "=" * 80)

### 2B. El Moderno Carismático con Herramientas Avanzadas

**Perfil**: Latino colombiano urbano, bilingüe, nomada digital

**Match**: Misma neurocientífica aventurera

**Hipótesis**: Las herramientas detectarán:
- Compatibilidad alta (estilos más alineados: aventura + modernidad)
- Recomendarán tono medio-atrevido para icebreaker
- Identificarán que humor autoirónico y Spanglish pueden funcionar

In [ ]:
# Obtener persona moderna
persona_moderno = get_persona_moderno_carismatico()

print("=" * 80)
print("ESCENARIO: Moderno Carismático -> Neurocientífica Aventurera")
print("=" * 80)
print(f"Persona: {persona_moderno['pais_origen']} - {persona_moderno['tipo_persona']}")
print(f"Match: {match_cientifica['nombre']} - {match_cientifica['profesion']}")
print("\n")

# Ejecutar ReAct
result_moderno_tools = run_react_with_coqueteo_tools(
    persona_dict=persona_moderno,
    match_dict=match_cientifica,
    verbose=True
)

#### Comparación: Romántico vs Moderno

Comparemos cómo las mismas herramientas generan outputs diferentes para cada arquetipo:

In [ ]:
print("\n" + "=" * 80)
print("COMPARACION: Romántico Clásico vs Moderno Carismático")
print("=" * 80)

rom_compat = result_romantico_tools['tool_results'].get('analizar_compatibilidad', {})
mod_compat = result_moderno_tools['tool_results'].get('analizar_compatibilidad', {})

print("\n1. COMPATIBILIDAD:")
print(f"   Romántico: {rom_compat.get('compatibilidad_porcentaje', 'N/A')}%")
print(f"   Moderno:   {mod_compat.get('compatibilidad_porcentaje', 'N/A')}%")

rom_ice = result_romantico_tools['tool_results'].get('generar_icebreaker', {})
mod_ice = result_moderno_tools['tool_results'].get('generar_icebreaker', {})

print("\n2. ICEBREAKER (Primera Opción):")
if rom_ice.get('opciones'):
    print(f"   Romántico: {rom_ice['opciones'][0]['texto'][:120]}...")
if mod_ice.get('opciones'):
    print(f"   Moderno:   {mod_ice['opciones'][0]['texto'][:120]}...")

rom_pred = result_romantico_tools['tool_results'].get('predecir_respuesta', {})
mod_pred = result_moderno_tools['tool_results'].get('predecir_respuesta', {})

print("\n3. PROBABILIDAD DE RESPUESTA:")
print(f"   Romántico: {rom_pred.get('probabilidad_respuesta', 'N/A')}%")
print(f"   Moderno:   {mod_pred.get('probabilidad_respuesta', 'N/A')}%")

print("\n" + "=" * 80)
print("INSIGHT CLAVE:")
print("Las herramientas especializadas permiten que el agente ReAct tome decisiones")
print("basadas en análisis cuantitativo en vez de generar outputs directamente.")
print("Esto resulta en estrategias más calibradas y explicables.")
print("=" * 80)

## Herramientas como Sistema Experto

### Por Qué Usar Herramientas en Vez de Generación Directa

**Approach 1: Generación Directa (sin herramientas)**
```python
prompt = "Genera un icebreaker para este perfil: {profile}"
response = llm.invoke(prompt)
```

**Problemas:**
- Output es caja negra (no sabemos cómo llegó a esa conclusión)
- Difícil de auditar o mejorar
- No puedes inyectar conocimiento de dominio
- Inconsistente entre runs

**Approach 2: ReAct con Herramientas (arquitectura actual)**
```python
tools = [analizar_compatibilidad, generar_icebreaker, predecir_respuesta]
agent = create_react_agent(llm, tools)
response = agent.invoke(profile)
```

**Ventajas:**
- **Explicabilidad**: Puedes ver qué herramientas se llamaron y por qué
- **Modularidad**: Puedes mejorar/reemplazar herramientas individuales
- **Conocimiento de dominio**: Las herramientas encapsulan heurísticas probadas
- **Auditabilidad**: Cada decisión tiene trace
- **Testing**: Puedes testear herramientas independientemente

### Cuándo Usar Herramientas

**Usa herramientas cuando:**
- La tarea requiere decisiones multi-paso basadas en datos
- Necesitas explicabilidad (qué se analizó, qué se decidió, por qué)
- Tienes conocimiento de dominio que quieres codificar (scoring, reglas de negocio)
- Quieres poder iterar en componentes individuales

**NO uses herramientas cuando:**
- La tarea es simple y directa (clasificación, extracción)
- No necesitas explicabilidad ni auditoría
- La latencia es crítica (cada tool call agrega ~1-3s)
- El modelo es suficientemente capaz de resolver en un paso

## Proximos Pasos

1. **Crea tus propias herramientas especializadas**: Define tools para tu dominio específico
2. **Implementa scoring cuantitativo**: Agrega métricas numéricas a tus herramientas (como compatibilidad_porcentaje)
3. **A/B testing de estrategias**: Compara herramientas simples vs complejas
4. **Implementa feedback loop**: Usa resultados reales para ajustar heurísticas de herramientas
5. **Monitorea tool usage**: Trackea qué herramientas se usan más y su tasa de éxito

**Comparación con CoT**: Mientras CoT es ideal para razonamiento explícito paso-a-paso, ReAct brilla cuando necesitas **ejecutar acciones** (buscar, calcular, validar) entre pasos de razonamiento.

---